In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('../input/train.csv')

In [ ]:
songs_df = pd.read_csv('../input/songs.csv')

In [ ]:
members_df = pd.read_csv('../input/members.csv')

In [ ]:
train_df.head()

msno - user ID, song_id - song ID, source_system_tab - name of the tab where the event triggered, source_screen_name - nme of the layout a user sees, source_type - entry point where user first plays music, target - 1 indicates recurring listening events

In [ ]:
train_df.info()

In [ ]:
train_df['source_system_tab'].nunique()

In [ ]:
train_df['source_screen_name'].nunique()

In [ ]:
train_df['source_type'].nunique()

So there are 9, 20 and 12 possibilities for the htree 'object' variables

Q1 - which is the most popular 'source system tab'

Are there any 'null' entries ??

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
train_df.isnull().aggregate('sum')

Is it important to fill the nulls here ? Or is it ok to live with it, if we just derive the probabilities ?? --- leaving it unfilled for now

In [ ]:
plt.figure(figsize=(14,7))
sns.countplot(x='source_system_tab', data=train_df, palette='RdBu_r')

In [ ]:
plt.figure(figsize=(14, 7))
sns.countplot(x='source_screen_name', data=train_df, palette='RdBu_r')

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x='source_type', data=train_df, palette='RdBu_r')

In [ ]:
temp_df = train_df.groupby(['source_system_tab', 'source_screen_name'])['target'].aggregate(['count', 'sum']).reset_index()

In [ ]:
temp_df['ratio'] = temp_df['sum']/temp_df['count']

In [ ]:
temp_df_pivot = temp_df.pivot('source_system_tab', 'source_screen_name', 'ratio')

In [ ]:
temp_df_pivot.head()

In [ ]:
plt.figure(figsize=(14, 7))
sns.heatmap(temp_df_pivot, annot=True)

In [ ]:
temp_df_pivot = temp_df.pivot('source_system_tab', 'source_screen_name', 'count')
plt.figure(figsize=(14,7))
sns.heatmap(temp_df_pivot)

Some combinations of screen name and system tab are more popular than others

In [ ]:
temp_df['count'].hist(bins=100)

most of the combinations have 'count' less than 100 --> is this significant enough for deriving a probability out of it ? i.e are these statistically significant ? 

In [ ]:
target_count_df = train_df.groupby(['source_system_tab', 'source_screen_name', 'source_type'])['target'].aggregate('count').reset_index()

In [ ]:
target_sum_df = train_df.groupby(['source_system_tab', 'source_screen_name', 'source_type'])['target'].aggregate('sum').reset_index()

In [ ]:
target_count_df['target_count'] = target_sum_df['target']

In [ ]:
target_count_df['ratio'] = target_count_df['target_count']/target_count_df['target']

In [ ]:
target_count_df.head()

In [ ]:
all3_cat_prob = target_count_df.copy()

In [ ]:
del target_count_df, target_sum_df

In [ ]:
all3_cat_prob.sort_values('target', ascending=False)

In [ ]:
source_system_df = train_df.groupby(['source_system_tab'])['target'].aggregate(['count', 'sum']).reset_index()

In [ ]:
source_system_df['ratio'] = source_system_df['sum']/source_system_df['count']

In [ ]:
source_system_df.head(20)

In [ ]:
source_screen_df = train_df.groupby(['source_screen_name'])['target'].aggregate(['count', 'sum']).reset_index()

In [ ]:
source_screen_df['ratio'] = source_screen_df['sum']/source_screen_df['count']

In [ ]:
source_screen_df.head()

In [ ]:
source_type_df = train_df.groupby(['source_type'])['target'].aggregate(['count', 'sum']).reset_index()

In [ ]:
source_type_df['ratio'] = source_type_df['sum']/source_type_df['count']

In [ ]:
source_type_df.head()

In [ ]:
combined_df = pd.merge(train_df, songs_df, on='song_id', how='inner')

In [ ]:
combined_df.head()

In [ ]:
combined_final_df = pd.merge(combined_df, members_df, on='msno', how='inner')

In [ ]:
combined_final_df.head()

In [ ]:
del combined_df, songs_df, members_df

In [ ]:
artistwise_target_df = combined_final_df.groupby('artist_name')['target'].aggregate(['count', 'sum']).reset_index()

In [ ]:
artistwise_target_df.sort_values('count', ascending=False).head(10)

In [ ]:
artistwise_target_df['ratio'] = artistwise_target_df['sum']/artistwise_target_df['count']

In [ ]:
artistwise_target_df[artistwise_target_df['ratio'] != 1.0].sort_values('ratio', ascending=False).head(10)

In [ ]:
langwise_target_df = combined_final_df.groupby('language')['target'].aggregate(['count', 'sum']).reset_index()
langwise_target_df['ratio'] = langwise_target_df['sum']/langwise_target_df['count']

In [ ]:
langwise_target_df.head(10)

In [ ]:
combined_final_df['song_length'].describe()

In [ ]:
combined_final_df.columns

In [ ]:
combined_final_df['registered_via'].unique()

In [ ]:
combined_final_df['registration_init_time'].nunique()

In [ ]:
combined_final_df['msno'].nunique()

In [ ]:
combined_final_df['expiration_date'].nunique()

In [ ]:
combined_final_df['song_id'].nunique()

Let's create a super simple dataset without the song and user IDs'

In [ ]:
trial_comb_df = combined_final_df.copy()

In [ ]:
trial_comb_df.columns

In [ ]:
    del trial_comb_df['msno']

In [ ]:
trial_comb_df.columns

In [ ]:
del trial_comb_df['song_id']

In [ ]:
trial_comb_df.isnull().aggregate('sum')/trial_comb_df.shape[0]

In [ ]:
del trial_comb_df['gender']
del trial_comb_df['composer']
del trial_comb_df['lyricist']

In [ ]:
trial_comb_df.isnull().aggregate('sum')/trial_comb_df.shape[0]

In [ ]:
trial_comb_df['language'].fillna('median', inplace=True)

In [ ]:
trial_comb_df['source_system_tab'].fillna('median', inplace=True)

In [ ]:
trial_comb_df['genre_ids'].fillna('median', inplace=True)

In [ ]:
trial_comb_df['source_type'].fillna('median', inplace=True)

In [ ]:
trial_comb_df['source_screen_name'].fillna('median', inplace=True)

In [ ]:
trial_comb_df.info()

In [ ]:
source_system_tab_dummies = pd.get_dummies(trial_comb_df['source_system_tab'], drop_first=True)

In [ ]:
source_screen_name_dummies = pd.get_dummies(trial_comb_df['source_screen_name'], drop_first=True)

In [ ]:
source_type_dummies = pd.get_dummies(trial_comb_df['source_type'], drop_first=True)

In [ ]:
genre_id_dummies = pd.get_dummies(trial_comb_df['genre_ids'], drop_first=True)

In [ ]:
del trial_comb_df['artist_name']

In [ ]:
trial_comb_df['expiration_date'].nunique()

In [ ]:
all3_cat_prob.info()

Super simple first cut solution - ignore song ID, user ID etc. Work with just the columns from the trial CSV file

In [ ]:
all3_cat_prob['target_prob'] = all3_cat_prob['target']/all3_cat_prob['target'].aggregate('sum')

In [ ]:
all3_cat_prob['target_count_prob'] = all3_cat_prob['target_count']/all3_cat_prob['target_count'].aggregate('sum')

In [ ]:
all3_cat_prob.head()

In [ ]:
overall_prob = all3_cat_prob['target_count'].aggregate('sum')/all3_cat_prob['target'].aggregate('sum')

In [ ]:
overall_prob

50% of the time there is an event

In [ ]:
test_df = pd.read_csv('../input/test.csv')

In [ ]:
test_df.head()

In [ ]:
del test_df['msno']

In [ ]:
del test_df['song_id']

Simple solution:Without info on song name etc - dumb solution !!

In [ ]:
test_df = pd.merge(test_df, all3_cat_prob, on =['source_system_tab', 'source_screen_name', 'source_type'], how='left')

In [ ]:
del test_df['target']

In [ ]:
del test_df['target_count']

In [ ]:
del test_df['target_prob']

In [ ]:
del test_df['target_count_prob']

In [ ]:
test_df.head()

In [ ]:
test_df.isnull().aggregate('sum')

In [ ]:
test_df['ratio'].fillna(0.5055, inplace=True)

In [ ]:
test_df.isnull().aggregate('sum')

In [ ]:
submit_df = pd.DataFrame()

In [ ]:
submit_df['id'] = test_df['id']
submit_df['target'] = test_df['ratio']

In [ ]:
submit_df.head()

In [ ]:
ids = test_df['id'].values

In [ ]:
targets = test_df['ratio'].values

In [ ]:
submit_df.to_csv('submission.csv', index=False, float_format='%.5f')

In [ ]:
temp = pd.read_csv('submission.csv')

In [ ]:
temp.head()